In [ ]:
import socket
from struct import pack

socket_file = "/run/postgresql/.s.PGSQL.5432"

In [ ]:
sock = socket.socket(socket.AF_UNIX, socket.SOCK_STREAM)
sock.connect(socket_file)
sock.settimeout(1)

In [ ]:
print(sock.send(b"???"))
print(sock.recv(1000))

## Encoding raw bytes

We are going to need to use `pack` from the `struct` package.

> `pack(fmt, v1, v2, ...) -> bytes`
> 
> Return a `bytes` object containing the values `v1`, `v2`, ... packed according to the format string `fmt`.

So, let's consider the right [format string](https://docs.python.org/3.5/library/struct.html#byte-order-size-and-alignment):

> [T]he first character of the format string can be used to indicate the byte order, size and alignment of the packed data, according to the following table:
> 
> | Character | Byte order             | Size     | Alignment |
> |:---------:| ---------------------- | -------- | --------- |
> | `@`         | native                 | native   | native    |
> | `=`         | native                 | standard | none      |
> | `<`         | little-endian          | standard | none      |
> | `>`        | big-endian             | standard | none      |
> | `!`        | network (= big-endian) | standard | none      |

Network seems like a good choice!

Then actual format characters can be chosen from [a long list](https://docs.python.org/3.5/library/struct.html#format-characters), but we know we only care about 32 bit (4 byte) and 16 bit (2 byte) integers:

> | Character | C Type    | Python type | Standard size
> |:---------:| :----------------------: | :--------: | :---------:|
> | ... |	... |	... |	... |
> | `h` |	short |	integer |	2 |
> | ... |	... |	... |	... |
> | `l` |	long |	integer |	4 |
> | ... |	... |	... |	... |

## Opening a connection

From the [Postgres documentation on message formats](https://www.postgresql.org/docs/current/static/protocol-message-formats.html):

**StartupMessage**

* `Int32`

    Length of message contents in bytes, including self.



* `Int32(196608)`

    The protocol version number. The most significant 16 bits are the major version number (3 for the protocol described here). The least significant 16 bits are the minor version number (0 for the protocol described here).

The protocol version number is followed by one or more pairs of parameter name and value strings. A zero byte is required as a terminator after the last name/value pair. Parameters can appear in any order. user is required, others are optional. Each parameter is specified as:

* `String`

    The parameter name. Currently recognized names are:

  * `user`
  
    The database user name to connect as. Required; there is no default.

  * `database`
    
    The database to connect to. Defaults to the user name.

  * `options`
    
    Command-line arguments for the backend. (This is deprecated in favor of setting individual run-time parameters.) Spaces within this string are considered to separate arguments, unless escaped with a backslash (\); write \\ to represent a literal backslash.

    In addition to the above, any run-time parameter that can be set at backend start time might be listed. Such settings will be applied during backend start (after parsing the command-line arguments if any). The values will act as session defaults.

* `String`

    The parameter value.

**Of course, it's never quite so simple.**

I ended up starting Wireshark and a connection over the loopback interface (`psql -h 127.0.0.1`) to figure out what the message format actually looks like. Strings are null (0x00) terminated, and in addition the startup message is terminated by a null byte (but other messages aren't).

In [ ]:
startup_message = b"user\0peitho\0\0"

# startup_params = {
#     b"user": b"peitho",
#     b"application_name": b"just messing around",
#     b"client_encoding": b"UTF8",
# }
# startup_message = b""
# for k, v in startup_params.items():
#     startup_message += k + b"\0" + v + b"\0"
# startup_message += b"\0"

version_string = pack("!l", 196608)
length = pack("!l", len(startup_message) + len(version_string) + 4)

sock.send(length + version_string + startup_message)  # Returns the number of bytes written to the socket

In [ ]:
data = sock.recv(1000) # Get (up to) 1000 bytes of data
data

In [ ]:
data.split(b"\0") # That's ugly, let's see if we can make it a little more readable...

## Making things easier

Let's build some utility functions for sending and receiving.

Most messages have this structure:

* Byte1

  Identifies the message type by a one-byte ASCII code.


* Int32
  
  Length of message contents in bytes, including self.


* Payload data (could be basically anything)

In [ ]:
def recv(n=1000):
    """Receive n (default 1000) bytes on the socket"""
    return list(filter(lambda x: x != b"", sock.recv(n).split(b"\0")))
#     return sock.recv(n).split(b"\0")

def send(identifier, msg):
    """Send a postgres identifier + length + message message. This adds the length field."""
    # The message length *excludes* the length of the code
    # but includes 4 bytes for the length of the message length field.
    
    print(identifier + pack("!l", 4 + len(msg) + 1) + msg + b"\0")  # for debugging
    return sock.send(identifier + pack("!l", 4 + len(msg)) + msg)

In [ ]:
send(b"Q", b"CREATE TEMPORARY TABLE my_temp_table (id SERIAL, data TEXT);\0")
recv()

In [ ]:
send(b"Q", b"""INSERT INTO my_temp_table (data) VALUES
    ('Oooh'),
    ('Ahhh'),
    ('RC presentations are pretty cool right? We get to share some awesome information on stuff we would never do outside of RC.');
\0""")
recv()

In [ ]:
send(b"Q", b"SELECT id, data FROM my_temp_table;\0")
recv()

## Paramaterized queries
**Parse**

* Byte1('P')

  Identifies the message as a Parse command.


* Int32
  
  Length of message contents in bytes, including self.


* String
  
  The name of the destination prepared statement (an empty string selects the unnamed prepared statement).


* String
  
  The query string to be parsed.


* Int16

  The number of parameter data types specified (can be zero). Note that this is not an indication of the number of parameters that might appear in the query string, only the number that the frontend wants to prespecify types for.

Then, for each parameter, there is the following:

* Int32
  
  Specifies the object ID of the parameter data type. Placing a zero here is equivalent to leaving the type unspecified.


**Bind**

* Byte1('B')

  Identifies the message as a Bind command.


* Int32
  
  Length of message contents in bytes, including self.


* String

  The name of the destination portal (an empty string selects the unnamed portal).


* String
  
  The name of the source prepared statement (an empty string selects the unnamed prepared statement).


* Int16
  
  The number of parameter format codes that follow (denoted C below). This can be zero to indicate that there are no parameters or that the parameters all use the default format (text); or one, in which case the specified format code is applied to all parameters; or it can equal the actual number of parameters.


* Int16[C]
  
  The parameter format codes. Each must presently be zero (text) or one (binary).


* Int16
  
  The number of parameter values that follow (possibly zero). This must match the number of parameters needed by the query.

Next, the following pair of fields appear for each parameter:

* Int32

  The length of the parameter value, in bytes (this count does not include itself). Can be zero. As a special case, -1 indicates a NULL parameter value. No value bytes follow in the NULL case.


* Byte ***n*** (*exactly n bytes*)
  
  The value of the parameter, in the format indicated by the associated format code. n is the above length.

After the last parameter, the following fields appear:

* Int16

  The number of result-column format codes that follow (denoted R below). This can be zero to indicate that there are no result columns or that the result columns should all use the default format (text); or one, in which case the specified format code is applied to all result columns (if any); or it can equal the actual number of result columns of the query.


* Int16[R]

  The result-column format codes. Each must presently be zero (text) or one (binary).

**Sync**

* Byte1('S')
  
  Identifies the message as a Sync command.


* Int32(4)

  Length of message contents in bytes, including self.


**ParseComplete** (*Response message type*)

* Byte1('1')

  Identifies the message as a Parse-complete indicator.


* Int32(4)
  
  Length of message contents in bytes, including self.


**ReadyForQuery** (*Response message type*)

* Byte1('Z')
  
  Identifies the message type. ReadyForQuery is sent whenever the backend is ready for a new query cycle.


* Int32(4) (*NB documentation claims this is 5 bytes but is lying*)

  Length of message contents in bytes, including self.


* Byte1

  Current backend transaction status indicator. Possible values are 'I' if idle (not in a transaction block); 'T' if in a transaction block; or 'E' if in a failed transaction block (queries will be rejected until block is ended).


In [ ]:
# Prepare a query
send(b"P",
     b"\0SELECT id, data FROM my_temp_table WHERE id = $1;\0" +
         pack("!h", 0))

# Bind paramter
send(b"B", 
     b"\0" +         # portal name
     b"\0" +         # query name
     pack("!h", 1) + # number of paramater format codes that follow
     pack("!h", 1) + # 1st paramter format code: 1 = binary, 0 = string (UTF8 encoding)
     pack("!h", 1) + # number of paramater values that follow
     pack("!l", 4) + # the length of the 1st paramter value (excluding this length specifier)
     pack("!l", 1) + # the 1st paramater value (1 as an integer)
     pack("!h", 0))  # the number of result column format codes that follow

# Execute the query
send(b"E", b"\0" + pack("!l", 0))

# Bind paramter
send(b"B", 
     b"\0" +         # portal name
     b"\0" +         # query name
     pack("!h", 1) + # number of paramater format codes that follow
     pack("!h", 1) + # 1st paramter format code: 1 = binary, 0 = string (UTF8 encoding)
     pack("!h", 1) + # number of paramater values that follow
     pack("!l", 4) + # the length of the 1st paramter value (excluding this length specifier)
     pack("!l", 3) + # the 1st paramater value (3 as an integer)
     pack("!h", 0))  # the number of result column format codes that follow

# Execute the query
send(b"E", b"\0" + pack("!l", 0))

# Sync and receive the data
send(b"S", b"")
recv()


In [ ]:
send(b"S", b"")
recv()

In [ ]:
send(b"X", b"") # Terminate
recv()

In [ ]:
sock.close()
del sock